<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_segmentation/blob/main/cloud_segmentation_cnn_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.6.0
!pip install keras~=2.6.0
!pip install tensorflow_addons
!pip install keras-segmentation
!pip install segmentation_models

     |████████████████████████████████| 458.3 MB 11 kB/s 
     |████████████████████████████████| 14.8 MB 32.3 MB/s 
     |████████████████████████████████| 132 kB 42.1 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=a081b00b2d02cd62b2760888c5d99488f6434cd57c55e1ac23eb46f4acb1fb65
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68723 sha256=3526394de9f277ae9ff28cee57743dc47c3efb739b52aa7448cc44d4d20bfba9
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: numpy
    Found existing installation: numpy 

     |████████████████████████████████| 1.3 MB 5.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 7.5 MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=29071 sha256=4f31b3f9b1819a8e628a6342c8e7d47150b7cd598f25df4075eea353e0c7789b
  Stored in directory: /root/.cache/pip/wheels/68/dc/b3/596a3e2461ba16e935ef31661c26e823f841cfb577cec4c47a
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires 

In [3]:
!cp -r "/content/drive/My Drive/img_satelite" "img_satelite"

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D ,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout,concatenate, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import segmentation_models as sm
import keras_segmentation
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
  
from keras.models import *
from keras.layers import *
sm.set_framework('tf.keras')

sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [23]:
path_img = 'img_satelite/wsiseg/allsky/'
path_mask = 'img_satelite/wsiseg/masks/'

Function to display images

In [17]:
def display(display_list,title=['Input Image', 'True Mask', 'Predicted Mask']):
  plt.figure(figsize=(15, 15))
  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]),cmap='magma')
    plt.axis('off')
  plt.show()

Hyperparameters

In [18]:
num_classes = 2
batch_size = 8
epochs = 20
height = 400
width = 400
channels = 3

# Pretrained model


In [19]:
model = sm.Unet('resnet34', encoder_weights='imagenet',classes=3, activation='softmax')# multiclass segmentation with non overlapping class masks (your classes + background)

Read the images

In [24]:
img_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)

mask_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)

In [27]:
train_image_generator =img_datagen.flow_from_directory(path_img,subset='training',seed = 909)

train_mask_generator = mask_datagen.flow_from_directory(path_mask,subset='training', seed = 909)

train_generator = zip(train_image_generator, train_mask_generator)

valid_image_generator =img_datagen.flow_from_directory(path_img,subset='validation', seed = 909)

valid_mask_generator = mask_datagen.flow_from_directory(path_mask,subset='validation',seed = 909)

valid_generator = zip(valid_image_generator, valid_mask_generator)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


Hyper-parameters

In [11]:
number_of_neurons = 100

Lets see some samples

In [12]:
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

In [15]:
model.fit(train_generator, steps_per_epoch=167,epochs=10)

Epoch 1/10
167/167 [==============================] - 4s 25ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 2/10
167/167 [==============================] - 3s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 3/10
167/167 [==============================] - 3s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 4/10
167/167 [==============================] - 3s 20ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 5/10
167/167 [==============================] - 3s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 6/10
167/167 [==============================] - 3s 20ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 7/10
167/167 [==============================] - 3s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 8/10
167/167 [==============================] - 3s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 9/10
167/167 [==============================] - 4s 21ms/step - loss: 0.0000e+00 - iou_score: 1.0000
Epoch 10/10
167/167 [=========================

In [ ]:
test_generator.classes

In [ ]:
y_pred

In [ ]:
import plotly.figure_factory as ff

y_true = test_generator.classes

z = confusion_matrix(y_true, y_pred)

x = list(test_generator.class_indices.keys())

y = list(test_generator.class_indices.keys())


In [ ]:
# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in z]

# set up figure 
fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()